# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

In [25]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import accuracy_score, f1_score


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("Churn.csv")

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


**Признаки**:  

>*RowNumber* — индекс строки в данных  
*CustomerId* — уникальный идентификатор клиента  
*Surname* — фамилия  
*CreditScore* — кредитный рейтинг  
*Geography* — страна проживания  
*Gender* — пол  
*Age* — возраст  
*Tenure* — количество недвижимости у клиента  
*Balance* — баланс на счёте  
*NumOfProducts* — количество продуктов банка, используемых клиентом  
*HasCrCard* — наличие кредитной карты  
*IsActiveMember* — активность клиента  
*EstimatedSalary* — предполагаемая зарплата  

**Целевой признак**:  
>Exited — факт ухода клиента  

In [4]:
df.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)

In [5]:
df.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0


**Доля объектов положительного класса:**

In [6]:
sum(df["Exited"]/len(df))

0.20369999999999389

**Доля объектов отрицательного класса:**

In [7]:
1-sum(df["Exited"]/len(df))

0.7963000000000061

>#### Объектов отрицательного класса намного больше

#### Onehot encoding

In [8]:
gender_one_hot = pd.get_dummies(df["Gender"], drop_first=True)
country_one_hot = pd.get_dummies(df["Geography"], drop_first=True)
df.drop(["Gender", "Geography"], axis=1, inplace=True)
df_one_hot = pd.concat([df, gender_one_hot, country_one_hot], axis=1)

#### Scaling

In [10]:
scaler = MinMaxScaler()
df_one_hot_scaled = df_one_hot.copy(deep=True)
df_one_hot_scaled[["Balance", "EstimatedSalary"]] = scaler.fit_transform(df_one_hot_scaled[["Balance", "EstimatedSalary"]])

#### Пропуски

In [12]:
df_one_hot_scaled.isna().sum()

CreditScore          0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Male                 0
Germany              0
Spain                0
dtype: int64

In [13]:
features = df_one_hot_scaled.drop("Exited", axis=1)
target = df["Exited"]

In [16]:
features.fillna(1, inplace=True)

#### Деление на обучающую, валидационную и тестовую выборки

In [36]:
features_train, features_test0, target_train, target_test0 = train_test_split(features, target, 
                                                                              test_size=0.2, 
                                                                              random_state=1)

In [37]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test0, target_test0, 
                                                                              test_size=0.5, 
                                                                              random_state=1)

In [18]:
features_train = pd.DataFrame(features_train)
features_valid = pd.DataFrame(features_valid)
target_train = pd.Series(target_train)
target_valid = pd.Series(target_valid)

## Обучение моделей

#### Logistic regression

In [19]:
lr = LogisticRegression()
lr.fit(features_train, target_train)

LogisticRegression()

In [46]:
predictions = lr.predict(features_valid)
print("Accuracy:", accuracy_score(predictions, target_valid))
print("F1:", f1_score(predictions, target_valid))

Accuracy: 0.843
F1: 0.34309623430962344


#### Random forest

In [32]:
rfc = RandomForestClassifier()
param_grid = { 
    'n_estimators': [10, 20, 30, 40, 50],
    'max_features': ['log2'],
    'max_depth' : [5,7,9,11,13,15],
    'criterion' : ['gini']
}

In [33]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV_rfc.fit(features_train, target_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini'],
                         'max_depth': [5, 7, 9, 11, 13, 15],
                         'max_features': ['log2'],
                         'n_estimators': [10, 20, 30, 40, 50]})

In [34]:
predictions = CV_rfc.best_estimator_.predict(features_valid)
print("accuracy:", accuracy_score(predictions, target_valid))
print("F1:", f1_score(predictions, target_valid))

accuracy: 0.866
F1: 0.5732484076433121


### Убираем дисбаланс

#### Upsampling

In [31]:
# Увеличиваем число наблюдений положительного класса
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [27]:
features_upsampled_train, target_upsampled_train = upsample(features_train, target_train, repeat=4)

In [29]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV_rfc.fit(features_upsampled_train, target_upsampled_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini'],
                         'max_depth': [5, 7, 9, 11, 13, 15],
                         'max_features': ['log2'],
                         'n_estimators': [10, 20, 30, 40, 50]})

In [41]:
predictions = CV_rfc.best_estimator_.predict(features_valid)
print("Accuracy:", accuracy_score(predictions, target_valid))
print("F1:", f1_score(predictions, target_valid))

Accuracy: 0.897
F1: 0.6227106227106227


#### Downsampling

In [40]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [42]:
features_downsampled_train, target_downsampled_train = downsample(features_train, target_train, fraction=0.5)

In [43]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV_rfc.fit(features_downsampled_train, target_downsampled_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini'],
                         'max_depth': [5, 7, 9, 11, 13, 15],
                         'max_features': ['log2'],
                         'n_estimators': [10, 20, 30, 40, 50]})

In [44]:
predictions = CV_rfc.best_estimator_.predict(features_valid)
print("Accuracy:", accuracy_score(predictions, target_valid))
print("F1:", f1_score(predictions, target_valid))

Accuracy: 0.872
F1: 0.6144578313253011


## Проверка на тестовой выборке

In [45]:
predictions = CV_rfc.best_estimator_.predict(features_test)
print("Accuracy:", accuracy_score(predictions, target_test))
print("F1:", f1_score(predictions, target_test))

Accuracy: 0.845
F1: 0.6191646191646192


># Вывод:
    - Нами был проведен первичный анализ данных, выявлен явный дисбаланс классов, отрицательного к положительному - 4 к 1.
    - Произвели предобработку данных, заполнили пропуски модой, прошкалировали количественные переменные, сделали One_hot_encoding. 
    - Модель случайного леса до борьбы с дисбалансом показывала результат метрики f1 - 57%.
    - Произвели upsampling положительного класса, благодаря чему удалось повысить f1 меру до 61%. Dowmsampling оказался более эффективен.
    - Проверка на тестовой выборке подтверждает, что downsampling помог справиться с дисбалансом классов и удалось увеличить метрику f1 до 62%.